In [31]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


In [32]:
dataset_train = pd.read_csv('NSE-TATAGLOBAL.csv')
training_set = dataset_train.iloc[:, 1:2].values # 选取开盘价

In [33]:
training_set

array([[234.05],
       [234.55],
       [240.  ],
       ...,
       [121.8 ],
       [120.3 ],
       [122.1 ]])

In [34]:
training_set.shape


(2035, 1)

In [35]:
training_set[1,0]

234.55

In [36]:
X_train = []
y_train = []
for i in range(60, 2035):
    X_train.append(training_set[i-60:i, 0])
    y_train.append(training_set[i, 0])   # 60个x 预测后面的一个y值
X_train, y_train = np.array(X_train), np.array(y_train)

In [37]:
y_train

array([271. , 262.7, 263. , ..., 121.8, 120.3, 122.1])

In [38]:
y_train.shape

(1975,)

In [39]:
X_train

array([[234.05, 234.55, 240.  , ..., 277.45, 281.  , 264.5 ],
       [234.55, 240.  , 233.3 , ..., 281.  , 264.5 , 271.  ],
       [240.  , 233.3 , 233.55, ..., 264.5 , 271.  , 262.7 ],
       ...,
       [130.1 , 134.85, 134.25, ..., 118.  , 117.6 , 120.1 ],
       [134.85, 134.25, 131.9 , ..., 117.6 , 120.1 , 121.8 ],
       [134.25, 131.9 , 134.5 , ..., 120.1 , 121.8 , 120.3 ]])

In [40]:
X_train.shape

(1975, 60)

In [41]:
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1)) # LSTM 输入(1975,60,1) 每次输入60个 输入1975次

In [42]:
X_train
X_train.shape

(1975, 60, 1)

In [43]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1))) # 
regressor.add(Dropout(0.2))

In [44]:
# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))  # 意味着将删除20%的层

In [45]:
# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [46]:
# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

In [47]:
# Adding the output layer
regressor.add(Dense(units = 1))# 指定1个单元的输出作为全连接层（Dense layer

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error') # adam优化器编译模型，并用均方误差（mean_squarred_error）来计算误差

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
1975/1975 [==============================] - 12s 6ms/step - loss: 21702.3769
Epoch 2/100
1975/1975 [==============================] - 9s 5ms/step - loss: 19968.5423
Epoch 3/100
1975/1975 [==============================] - 10s 5ms/step - loss: 19075.4919
Epoch 4/100
1975/1975 [==============================] - 11s 5ms/step - loss: 18263.2865
Epoch 5/100
1975/1975 [==============================] - 10s 5ms/step - loss: 17478.4020
Epoch 6/100
1975/1975 [==============================] - 9s 5ms/step - loss: 16749.9428
Epoch 7/100
1975/1975 [==============================] - 9s 5ms/step - loss: 16041.1750
Epoch 8/100
1975/1975 [==============================] - 9s 5ms/step - loss: 15372.8129
Epoch 9/100
1975/1975 [==============================] - 9s 5ms/step - loss: 14735.3448
Epoch 10/100
1975/1975 [==============================] - 9s 5ms/step - loss: 14075.9433
Epoch 11/100
1975/1975 [==============================] - 10s 5ms/step - loss: 13450.4566
Epoch 12/100
1975/1975 [=

KeyboardInterrupt: 

In [52]:
regressor.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 60, 50)            10400     
_________________________________________________________________
dropout_5 (Dropout)          (None, 60, 50)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 60, 50)            20200     
_________________________________________________________________
dropout_6 (Dropout)          (None, 60, 50)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 60, 50)            20200     
_________________________________________________________________
dropout_7 (Dropout)          (None, 60, 50)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 50)                20200     
__________

In [55]:
# Getting the real stock price of 2017
dataset_test = pd.read_csv('tatatest.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))

In [56]:
# Getting the predicted stock price of 2017
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 76):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

NotFittedError: This MinMaxScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.